<a href="https://colab.research.google.com/github/pabloalfaro/drl-grid/blob/main/pruebas/dqn/DQN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Deep Q-Learning

Comprobar si está ejecutando en Google Colaboratory.

In [1]:
!pip install torch torchvision pillow gym requests sklearn matplotlib collections-extended numpy box2d-py gym[Box_2D]

     |████████████████████████████████| 448 kB 6.3 MB/s 


In [2]:
import sys
IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    from google.colab import drive
    drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import sys
IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    %cd "/content/drive/MyDrive/dqn"
    render = False
else:
    render = True

/content/drive/MyDrive/dqn


Importación de la librería gym:

In [4]:
import gym

Implementación de un agente con comportamiento sin entrenar utilizando la clase Agent que se encuentra en el archivo dqn_agent.py.

In [5]:
from dqn_agent import Agent

agent = Agent(state_size=8, action_size=157, seed=0)

## Algoritmo
A continuación se proporciona una implementación genérica del algoritmo Deep Q-Learning (DQN).

In [23]:
import gym
import random
import torch
import numpy as np
from collections import deque
import matplotlib.pyplot as plt
from pprint import pprint
%matplotlib inline

In [25]:
import grid2op
from grid2op.Converter import IdToAct

In [48]:
def dqn(env,n_episodes=2000, max_t=1000, eps_start=1.0, eps_end=0.01, eps_decay=0.995):
    """Deep Q-Learning.
    
    Params
    ======
        n_episodes (int): numero maximo de episodios de entrenamiento (n_episodios)
        max_t (int): numero maximo de pasos por episodio (n_entrenamiento)
        eps_start (float): valor inicial de epsilon
        eps_end (float): valor final de epsilon
        eps_decay (float): factor de multiplicacion (por episodio) de epsilon
    """
    scores = []                        # puntuaciones de cada episodio
    scores_window = deque(maxlen=100)  # puntuaciones de los ultimos 100 episodios
    eps = eps_start                    # inicializar epsilon
    
    for i_episode in range(1, n_episodes+1):
        state = env.reset()
        #pprint(state) 
        # El estado es un diccionario y además es demasiado grande para tu red (tiene solo 8 neuronas de entrada según lo que has escrito)
        # Esto son 8 datos cualquiera que he cogido del diccionario, esto deberías de cambiarlo por aquellos datos que conformen tu problema realmente,
        # es solo para que se lo trague la red y veamos que funciona
        state=state['a_ex'][0:8] # --> Esto funciona :)
        score = 0
        
        for t in range(max_t):
            
            # elegir accion At con politica e-greedy
            # print(state)
            action = agent.act(state, eps)
            print('Acción devuelta:', action)

            # create the converter
            converter = IdToAct(env.action_space)
            converter.init_converter(redispatch=False)
            
            gym_action = converter.convert_act(action)  # this represents the same action
            print(gym_action)

            # El problema está en que debe ser un diccionario
            # action = env.action_space.to_gym(action[0])
            # action = gym.spaces.Dict()

            # action_dict = {
            # "change_bus": action[0][0:56],
            # "change_line_status": action[0][56:76],
            # "redispatch": action[0][76:81],
            # "set_bus": action[0][81:137],
            # "set_line_status": action[0][137:157]}
            #El siguiente problema que se plantea es que el action devuelto por la red es un único valor.
            
            # aplicar At y obtener Rt+1, St+1
            # next_state, reward, done, _ = env.step(action_dict)
            next_state, reward, done, _ = env.step(action)

            next_state=next_state['a_ex'][0:8]
            print(next_state)
            # He mirado el action space del entorno ya adaptado a gym y se esperan 5 datos, sospecho que puede venir de como se usa la librería torch
            # Por eso falla env.action()
            
            # almacenar <St, At, Rt+1, St+1>
            agent.memory.add(state, action, reward, next_state, done)
            
            # train & update
            agent.step(state, action, reward, next_state, done)
            
            # avanzar estado
            state = next_state
            score += reward
            
            if done:
                break 

        scores_window.append(score)       # guardar ultima puntuacion
        scores.append(score)              # guardar ultima puntuacion
        eps = max(eps_end, eps_decay*eps) # reducir epsilon
        
        print('\rEpisodio {}\tPuntuacion media (ultimos {:d}): {:.2f}'.format(i_episode, 100, np.mean(scores_window)), end="")
        if i_episode % 100 == 0:
            print('\rEpisodio {}\tPuntuacion media ({:d} anteriores): {:.2f}'.format(i_episode, 100, np.mean(scores_window)))
        if np.mean(scores_window)>=200.0:
            print('\nProblema resuelto en {:d} episodios!\tPuntuacion media (ultimos {:d}): {:.2f}'.format(i_episode-100, 100, np.mean(scores_window)))
            torch.save(agent.qnetwork_local.state_dict(), 'checkpoint.pth') # guardar pesos de agente entrenado
            break
    return scores

# scores = dqn()

# # plot the scores
# fig = plt.figure()
# ax = fig.add_subplot(111)
# plt.plot(np.arange(len(scores)), scores)
# plt.ylabel('Puntuacion')
# plt.xlabel('Episodio #')
# plt.show()

---
## Trabajo de grid2op
---

In [8]:
 !pip install grid2op[optional]  # Para ejecutar el código en Colab

     |████████████████████████████████| 12.1 MB 4.3 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 6.0 MB 23.5 MB/s 
     |████████████████████████████████| 3.4 MB 40.4 MB/s 
     |████████████████████████████████| 1.5 MB 42.6 MB/s 
     |████████████████████████████████| 130 kB 57.5 MB/s 
     |████████████████████████████████| 623 kB 52.6 MB/s 
     |████████████████████████████████| 280 kB 40.3 MB/s 
     |████████████████████████████████| 4.3 MB 38.7 MB/s 
     |████████████████████████████████| 428 kB 36.7 MB/s 
  Created wheel for grid2op: filename=Grid2Op-1.7.0-py3-none-any.whl size=12476815 sha256=16bc32194aa383b39a93da9bf3e068a58f829bff5d907640047f6cea52e79ba6
  Stored in directory: /root/.cache/pip/wheels/03/af/ba/bd0f4f3138c2b43a9468c413ad6f8360ab33235d33b09ef9d5
  Created wheel for pandapower: filename=pandapower-2.9.0-py3-none-any.whl size=5931109 sh

In [9]:
 !pip install jyquickhelper # Para ejecutar el código en Colab

In [10]:
import os
import sys
import grid2op

In [11]:
# import the usefull classes
from pprint import pprint
import numpy as np
import shutil
from tqdm.notebook import tqdm  # for easy progress bar

from grid2op import make
from grid2op.Agent import DoNothingAgent

from grid2op.Reward import GameplayReward, L2RPNReward
from grid2op.gym_compat import GymObservationSpace


max_iter = 50 # Se consideran 50 iteraciones para que sea más rápido
train_iter = 50
max_eval_step = 20
env_name = "rte_case14_redisp"
env = make(env_name, test=False, reward_class=L2RPNReward)
env.seed(0)  # Una semilla para que los agentes sean reproducibles
#pprint(env.observation_space.__dict__)
print(len(env.observation_space.__dict__['shape']))
print(len(env.observation_space.__dict__['dtype']))
print(len(env.observation_space.__dict__['attr_list_vect']))

/usr/local/lib/python3.7/dist-packages/grid2op/MakeEnv/Make.py:413: UserWarning: It is the first time you use the environment "rte_case14_redisp".
We will attempt to download this environment from remote
  warnings.warn(_MAKE_FIRST_TIME_WARN.format(dataset_name))


downloading the training data, this may take a while.


rte_case14_redisp.tar.bz2: 292MB [01:01, 4.76MB/s]                           


Extract the tar archive in "/root/data_grid2op"
	 Successfully updated file "grid.json" for environment "rte_case14_redisp"
You may now use the environment "rte_case14_redisp" with the available data by invoking:
	env = grid2op.make("rte_case14_redisp")
51
51
51


In [12]:
from grid2op.Reward import L2RPNReward

from grid2op.gym_compat import GymEnv, MultiDiscreteActSpace

In [49]:
# Convertimos el entorno a uno del estándar gym con un espacio de observaciones y acciones entendible.
gym_env = GymEnv(env)

# Por si quereis sacar los espacios por pantalla:
# print(gym_env.observation_space)
# print(gym_env.action_space)
scores = dqn(env=gym_env)

# plot the scores
fig = plt.figure()
ax = fig.add_subplot(111)
plt.plot(np.arange(len(scores)), scores)
plt.ylabel('Puntuacion')
plt.xlabel('Episodio #')
plt.show()

Acción devuelta: 79


AttributeError: ignored

Prueba tranformando el entorno

In [ ]:
!pip install git+https://github.com/DLR-RM/stable-baselines3

In [ ]:
from grid2op.Reward import L2RPNReward

from grid2op.gym_compat import GymEnv, MultiDiscreteActSpace

from stable_baselines3.common.policies import BasePolicy
from stable_baselines3 import A2C, PPO
from stable_baselines3.common.env_checker import check_env
from pprint import pprint #Para ver los diccionarios con una estructura más elegante de la que ofrece print convencional

In [ ]:
gym_env = GymEnv(env)
# Aunque el espacio de observaciones y acciones ya es correcto, Stable Baselines 3 no tiene soporte para espacio de acciones mixtos basados en diccionarios
# Por lo que no nos queda más remedio que discretizar todo el espacio de acciones (leer docstring de https://github.com/rte-france/Grid2Op/blob/master/grid2op/gym_compat/multidiscrete_gym_actspace.py)
print("Espacio de acciones original: {}".format(gym_env.action_space))
gym_env.action_space = MultiDiscreteActSpace(gym_env.init_env.action_space)
print("Espacio de acciones discretizado: {}".format(gym_env.action_space))

In [ ]:
check_env(gym_env)

In [ ]:
env = gym_env

In [ ]:
prueba = env.reset()
data = list(prueba.items())
an_array = np.array(data)

an_array

In [ ]:
scores = dqn(env=gym_env)

In [51]:
import grid2op
from grid2op.Converter import IdToAct
env = grid2op.make()
converter = IdToAct(env.action_space)

# the path were will save it
path_ = "/path/where/it/is/saved/"
name_file = "tmp_convert.npy"

# init the converter, the first time, here by passing some key word arguments, to not consider
# redispatching for example
converter.init_converter(redispatch=False)


# i just do an action, for example the number 27... whatever it does does not matter here
act = converter.convert_act(27)

print(act)

This action will:
	 - NOT change anything to the injections
	 - NOT perform any redispatching action
	 - NOT modify any storage capacity
	 - NOT perform any curtailment
	 - Force reconnection of 1 powerlines ([6])
	 - NOT switch any line status
	 - NOT switch anything in the topology
	 - Set the bus of the following element(s):
	 	 - Assign bus 1 to line (origin) id 6 [on substation 3]
	 	 - Assign bus 1 to line (extremity) id 6 [on substation 4]
